In [6]:
import os
import numpy
import pandas as pd
from sqlalchemy import create_engine
import csv
import json

In [7]:
# reading in data
#  benchmark 1: Fetch / download / retrieve a remote data file by URL
movie_data = pd.read_csv("/Users/Anna/Downloads/Highest Holywood Grossing Movies.csv")
movie_data.head(5)

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN


In [8]:
# create database
host_name = "localhost"
host_ip = "127.0.0.1"
port = "3306"
user_id = "ahs2aew"
pwd = "$w1mm3r!"

src_dbname = "movies"

In [9]:
# save data into new database
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        sqlEngine.execute(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});")
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()


In [10]:
# changing data from CSV --> JSON, then JSON --> 
# benchmark 2: Convert the general format and data structure of the data source 

# the function to convert CSV --> JSON takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath):
    data = {} # making a dictionary
    with open (csvFilePath, encoding = 'utf-8') as csvf: # open a CSV reader called DictReader
        csvReader = csv.DictReader(csvf)
        for rows in csvReader: # convert each row into a dictionary and add it into the data
            key = rows['Title']# assuming a column named 'No' to be primary key (??? -- from website instructions)
            data[key] = rows
        # open a json writer, and use json.dumps() function to dump data
        with open (jsonFilePath, 'w',encoding = 'utf-8') as jsonf: 
            jsonf.write(json.dumps(data,indent =4))
            
# decide the 2 file paths based on your system
csvFilePath = "/Users/Anna/Downloads/Highest Holywood Grossing Movies.csv"
jsonFilePath = "/Users/Anna/Downloads/Highest Holywood Grossing Movies.json"

# call the make_json function
make_json(csvFilePath, jsonFilePath)

In [11]:
# now that we have converted the file from CSV to JSON, we can convert it back to a CSV
with open('/Users/Anna/Downloads/Highest Holywood Grossing Movies.json', encoding='utf-8') as inputfile:
    df = pd.read_json(inputfile)

df.to_csv('/Users/Anna/Downloads/Highest Holywood Grossing Movies.csv', encoding='utf-8', index=False)

In [12]:
# change number of columns: drop or add
# benchmark 3: Modify the number of columns from the source to the destination, reducing or adding columns. 



# going forward, we can use movie_data which is our CSV file 
# could make a new column: what percentage of total sales are domestic (from the US)?
# would divide domestic sales / world sales 
# new column name: % of Sales from US 

movie_data.head(5)
#movie_data = movie_data[,-[1]]
#movie_data.head(5)


#df = df.assign(newcol = 

# could take out the first column because it serves no purpose
#pd.DataFrame.drop(labels=None, axis=1, inplace=True)

#new_movie_data = movie_data.drop(movie_data.columns[1], axis=1, inplace=True) # prob don't run more than once
#new_movie_data.head(5)
#  something got messed up here...

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN


In [13]:
# trying to create new column that's a transformation of existing columns
movie_data["Percentage Sales Domestic"] = (movie_data["Domestic Sales (in $)"] / movie_data["World Sales (in $)"]) * 100
movie_data['Percentage Sales Domestic'] = round(movie_data['Percentage Sales Domestic'])
movie_data.head(5)


,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,Percentage Sales Domestic
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,45.0
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,31.0
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,27.0
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,52.0
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,33.0


In [14]:
movie_data.head(5)
# have to remove old percentage_domestic_sales column

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,Percentage Sales Domestic
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,45.0
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,31.0
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,27.0
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,52.0
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,33.0


In [15]:
movie_data.head(5)

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,Percentage Sales Domestic
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,45.0
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,31.0
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,27.0
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,52.0
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,33.0


In [16]:
# make another column for percentage sales international, also rounded
movie_data['Percentage Sales International'] = (movie_data["International Sales (in $)"] / movie_data["World Sales (in $)"]) * 100
movie_data['Percentage Sales International'] = round(movie_data['Percentage Sales International'])
movie_data.head(5)

,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,Percentage Sales Domestic,Percentage Sales International
0,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,45.0,55.0
1,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,31.0,69.0
2,2,Avatar (2009),A paraplegic Marine dispatched to the moon Pan...,Twentieth Century Fox,"December 16, 2009",760507625,2086738578,2847246203,"['Action', 'Adventure', 'Fantasy', 'Sci-Fi']",2 hr 42 min,PG-13,27.0,73.0
3,3,Black Panther (2018),"T'Challa, heir to the hidden but advanced king...",Walt Disney Studios Motion Pictures,NaN,700426566,647171407,1347597973,"['Action', 'Adventure', 'Sci-Fi']",2 hr 14 min,NaN,52.0,48.0
4,4,Avengers: Infinity War (2018),The Avengers and their allies must be willing ...,Walt Disney Studios Motion Pictures,NaN,678815482,1369544272,2048359754,"['Action', 'Adventure', 'Sci-Fi']",2 hr 29 min,NaN,33.0,67.0


In [17]:
# adding a primary key 
movie_data.insert(0, "key", range(1, movie_data.shape[0]+1))


In [18]:
movie_data.head(2)

,key,Unnamed: 0,Title,Movie Info,Distributor,Release Date,Domestic Sales (in $),International Sales (in $),World Sales (in $),Genre,Movie Runtime,License,Percentage Sales Domestic,Percentage Sales International
0,1,0,Star Wars: Episode VII - The Force Awakens (2015),"As a new threat to the galaxy rises, Rey, a de...",Walt Disney Studios Motion Pictures,"December 16, 2015",936662225,1132859475,2069521700,"['Action', 'Adventure', 'Sci-Fi']",2 hr 18 min,PG-13,45.0,55.0
1,2,1,Avengers: Endgame (2019),After the devastating events of Avengers: Infi...,Walt Disney Studios Motion Pictures,"April 24, 2019",858373000,1939128328,2797501328,"['Action', 'Adventure', 'Drama', 'Sci-Fi']",3 hr 1 min,PG-13,31.0,69.0


In [19]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)

sqlEngine.execute(f"DROP DATABASE IF EXISTS `{src_dbname}`;")
sqlEngine.execute(f"CREATE DATABASE `{src_dbname}`;")
sqlEngine.execute(f"USE {src_dbname};")

OperationalError: (pymysql.err.OperationalError) (1045, "Access denied for user 'ahs2aew'@'localhost' (using password: YES)")
(Background on this error at: http://sqlalche.me/e/14/e3q8)

In [20]:
# write it into SQL database
#db_operation = "insert"

#tables = [('dim_customers', df_customers, 'customer_key'),
         # ('dim_employees', df_employees, 'employee_key'),
         # ('dim_products', df_products, 'product_key'),
         # ('dim_shippers', df_shippers, 'shipper_key')]

user_id = "root"

primary_key = movie_data['key']
db_operation = 'insert'

#for table_name, dataframe, primary_key in tables:
set_dataframe(user_id, pwd, host_name, src_dbname, movie_data, 'hello', primary_key, db_operation)
# don't have a table name?? -- tried taking out table name argument 
# db_operation

    

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '0        1\n1        2\n2        3\n3        4\n4        5\n      ... \n913    914\n914' at line 1")
[SQL: ALTER TABLE hello ADD PRIMARY KEY (0        1
1        2
2        3
3        4
4        5
      ... 
913    914
914    915
915    916
916    917
917    918
Name: key, Length: 918, dtype: int64);]
(Background on this error at: http://sqlalche.me/e/14/f405)

str

In [21]:
# brief summary of data
# Benchmark 5: Generate a brief summary of the data file ingestion

movie_data.describe()
# here, we can see an overview of the data 
# there are 918 observations. 
len(movie_data) # here, we get the number of records
# number of columns:
col = movie_data.columns
len(col)
# There are 14 columns 

14

In [ ]:
# Documentation: 
# First, we get the data. I got mine as a CSV from Kaggle.com. After getting the data, we can change the format of it: 
# for example, I changed the CSV to a JSON file, and then back to CSV to show that you can
# relatively easily switch between file formats. 
# After that, I manipulated the number of columns. For this dataset, I created 2 new columns to show percentage of movie
# sales that came from domestic sales, and then from internatinoal sales. This helped to make it more clear
# where the movie was selling the most. This could help people working on a movie determine where to focus
# their marketing efforts, to give one example of a use for these new columns. You could also see if, for a majority
# of these highest grossing movies, do more sales tend to come from domestic (in US) markets or international ones --
# you could then use a machine learning algorithm to potentially predict the success of a movie based on how well it does
# in international vs. domestic markets. 
